TODO:
    - fix candidate names 
    - add rest of info
    - check magnitude uncertainties with declan 
    - DONE: merge of candidates, district, party, office

In [1]:
import pandas as pd
import numpy as np
import os

path = 'raw'
os.chdir(path)

df = pd.read_csv('2020 General Election Precinct-Level Results.csv', encoding='latin-1')

In [2]:
df = df.drop([0, 1, 182871])
df = df.rename(columns={"2020 Statewide General Election Results": "county_name", "Unnamed: 1": "precinct", "Unnamed: 2": "race", "Unnamed: 3": "candidate", "Unnamed: 4": "votes"})

In [3]:
cand = pd.read_excel('candidate_party_list.xlsx', index_col=None)  
cand = cand.drop([27, 55, 83, 111, 139, 167, 195, 223, 251, 279, 307, 335, 362, 390, 418, 446, 474, 502, 530])
cand = cand.rename(columns={"Name as it Should Appear on the Ballot               (Last Name First)": "candidate"})
cand['candidate'] = cand['candidate'].str.upper()
cand = cand.drop(['Filing Office', 'Last Name', 'First Name', 'Notes'], axis=1)
cand = cand.replace(np.nan, '', regex = True)

In [4]:
df['candidate'] = df['candidate'].str.upper()

In [5]:
df['candidate'] = df['candidate'].replace({'BOSTA, JOHN': 'BOSTA, JOHN F.', 
                                           'HANSEN, ALEXIS': 'HANSEN, ALEXIS M.',
                                           'HAFEN, II, GREGORY T.': 'HAFEN II, GREGORY T.',
                                           'HAFEN II, GREGORY': 'HAFEN II, GREGORY T.',
                                           'RUBINSON, BARRY': 'RUBINSON, BARRY L.',
                                           'BUCK, CARRIE': 'BUCK, CARRIE ANN',
                                           'CANNIZZARO, NICOLE': 'CANNIZZARO, NICOLE JEANETTE',
                                           'DURAN, BEATRICE': 'DURAN, BEATRICE A.',
                                           'JONES, TIERRA': 'JONES, TIERRA D.',
                                           'HAFEN II, GREGORY': 'HAFEN, II, GREGORY T.',
                                           'BURTON, REBECCA': 'BURTON, REBECCA L.',
                                           'JOHNSON, SUSAN': 'JOHNSON, SUSAN HOLLAND',
                                           'KISHNER, JOANNA': 'KISHNER, JOANNA S.',
                                           'BELLONA, WM "JOHN"': 'WM "JOHN" BELLONA',
                                           'STURMAN, GLORIA': 'STURMAN, GLORIA J.',
                                           'ALLF, NANCY': 'ALLF, NANCY L.',
                                           'GREGORY, THOMAS W.': 'THOMAS W GREGORY',
                                           'YOUNG, TOD': 'TOD YOUNG',
                                           'CAFFERATA-JENKINS, CAREN': 'CAREN CAFFERATA-JENKINS',
                                           'MONTERO, MICHAEL': 'MONTERO, MICHAEL R.',
                                           'ABERASTURI, LEON': 'ABERASTURI LEON',
                                           'STEINHEIMER, CONNIE': 'STEINHEIMER, CONNIE J.',
                                           'PETERS, SARAH': 'PETERS, SARAH C',
                                           'GANSERT, HEIDI S.': 'GANSERT, HEIDI SEEVERS',
                                           'WARD, JENNIFER M.': 'JENNIFER M. WARD',
                                           'GILES, MICHAEL R.': 'MICHAEL R. GILES',
                                           'PORTMAN, JOHN J.T.': 'JOHN J.T. PORTMAN',
                                           'PILON, MARK W.': 'MARK W. PILON',
                                           'MACHADO, KEVIN E.': 'KEVIN E. MACHADO',
                                           'LIST, JAMES "JIM"': 'JAMES "JIM" LIST',
                                           'KNIGHT, SHERRY LYNN': 'SHERRY LYNN KNIGHT',
                                           'CARRUTH, MARCI': 'MARCI CARRUTH',
                                           'RACKLEY, LARRY': 'LARRY RACKLEY',
                                           'HUDSON, SHAYLA': 'SHAYLA HUDSON'})

In [6]:
df = pd.merge(df, cand, on = ['candidate'], 
             how = 'left')

In [ ]:
not in df at all: 
PAUL R. SCHLEHR
GIOCOECHEA, JJ

In [7]:
df['district'] = df['race']

In [8]:
df = df.rename(columns={"Party Affiliation": "party"})


In [9]:
df = df.replace(np.nan, '', regex = True)

In [10]:
df['party'] = df['party'].replace({'': 'NO PARTY'})

In [11]:
df['party_detailed'] = df['party'].replace({'NO POLITICAL PARTY': 'INDEPENDENT',
                                                     'IAP': 'INDEPENDENT',
                                                     'Independent': 'INDEPENDENT',
                                                     'INDEPENDENT AMERICAN': 'INDEPENDENT',
                                                     'Independent American': 'INDEPENDENT',
                                                     'NonPartisan': 'NONPARTISAN',
                                                     'Non-Partisan': 'NONPARTISAN',
                                                     'NON-PARTISAN': 'NONPARTISAN',
                                                     'NP': 'NONPARTISAN',
                                                     'Nonpartisan': 'NONPARTISAN',
                                                     'Democrat': 'DEMOCRAT',
                                                     'DEMOCRATIC': 'DEMOCRAT',
                                                     'DEM': 'DEMOCRAT',
                                                     'Democratic': 'DEMOCRAT',
                                                     'Libertarian': 'LIBERTARIAN',
                                                     'Republican': 'REPUBLICAN',
                                                     'REP': 'REPUBLICAN'})


In [12]:
df.loc[((df.candidate == 'NO') & (df.party_detailed == 'NO PARTY')), 'party_detailed'] = ''
df.loc[((df.candidate == 'YES') & (df.party_detailed == 'NO PARTY')), 'party_detailed'] = ''

df.loc[df.party_detailed == 'NO PARTY', 'party_detailed'] = 'NONPARTISAN'

In [13]:
df['party_simplified'] = df['party_detailed'].replace({'INDEPENDENT': 'OTHER'})

In [14]:
df = df.drop(['Office Sought', 'party'], axis=1)

***MAGNITUDE***

'TOPAZ RANCH ESTATES GENERAL IMPROVEMENT DISTRICT (One, 4 Year Term and One 2 Year-Unexpired Term)' (mag 2?)
'SKYLAND GENERAL IMPROVEMENT DISTRICT (Three, 4 Year Terms and Two, 2 Year-Unexpired Terms)' (mag 5?)
Elko City Council, 2 Seats (may want to check for magnitude change to 2)
Carlin City Council, 2 Seats (mag 2? )
WELLS CITY COUNCIL, 3 SEATS 
WEST WENDOVER CITY COUNCIL, SEATS 3 AND 4

SCHOOL DISTRICT TRUSTEE  **SEAT 1A, 1B, 1C, D**
CITY COUNCIL, SEAT 5
COUNTY COMMISSIONER **SEAT A, B, 2, 3, 4**
PERSHING COUNTY SCHOOL DISTRICT TRUSTEE, SEAT E
RENO CITY COUNCIL, AT-LARGE
HOSPITAL DISTRICT TRUSTEE **SEAT 3, 5**

In [15]:
def cleanDistrict(x):
    if ',' in x and '(' not in x:  
        index = x.index(',')
        return x[index+2:]
    else: 
        if 'President' in x or 'QUESTION' in x:
            return 'STATEWIDE'
        if 'District' in x and ('Water District' not in x and 'Improvement District' not in x and 'Sanitation District' not in x): 
            if 'Overton Power District 5 Mesquite' in x: 
                return '5'
            else: 
                index = x.index('District')
                return x[index+9]
        if 'DISTRICT' in x and 'DISTRICT DIRECTOR' not in x and 'WATER DISTRICT TRUSTEE' not in x:
            if 'SEAT' in x:
                index = x.index('SEAT')
                return x[index:]
            else: 
                index = x.index('DISTRICT')
                return x[index+9:]
        if 'Department' in x or 'DEPARTMENT' in x or 'SEAT' in x or 'Seat' in x: 
                if 'Department' in x: 
                    index = x.index('Department')
                    return x[index:]
                elif 'DEPARTMENT' in x: 
                    index = x.index('DEPARTMENT')
                    return x[index:]
                elif 'SEAT' in x:
                    index = x.index('SEAT')
                    return x[index:]
                else: 
                    index = x.index('Seat')
                    return x[index:]
        else: 
            return ''

In [16]:
df['district'] = df['district'].apply(cleanDistrict)

In [17]:
def cleanDistrict2(x):
    if len(x) <=11 and ('District' in x or 'DISTRICT' in x):
        x = x[-2:].strip()
        if any(char.isdigit() for char in x):
            return x.zfill(3)
        else:
            return x
    elif 'Dist.' in x:
        x = x[-1:]
        return x.zfill(3)
    else: 
        if '(One, 4 Year Term and One 2 Year-Unexpired Term)' in x or '(Three, 4 Year Terms and Two, 2 Year-Unexpired Terms)' in x:
            return ''
        elif 'CITY OF FERNLEY' in x:
            index = x.index('WARD')
            return x[index:]
        else:
            return x

In [18]:
df['district'] = df['district'].apply(cleanDistrict2)

In [19]:
df['magnitude'] = 1
df.loc[df['district'].str.contains('2 Seats'), 'magnitude'] = 2
df.loc[df['district'].str.contains('3 SEATS'), 'magnitude'] = 3
df.loc[df['district'].str.contains('SEATS 3 AND 4'), 'magnitude'] = 2

In [20]:
df.loc[df['race'].str.contains('TOPAZ RANCH ESTATES GENERAL IMPROVEMENT DISTRICT (One, 4 Year Term and One 2 Year-Unexpired Term)'), 'magnitude'] = 2
df.loc[df['race'].str.contains('SKYLAND GENERAL IMPROVEMENT DISTRICT (Three, 4 Year Terms and Two, 2 Year-Unexpired Terms)'), 'magnitude'] = 5

/Users/jennifermiranda/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [21]:
df["district"] = df["district"].str.replace('Dept.', 'Department', regex=True)

df['district'] = df['district'].replace({"2 Seats": ""})
df['district'] = df['district'].replace({"3 SEATS": ""})
df['district'] = df['district'].replace({"SEATS 3 AND 4": ""})

In [22]:
df["candidate"] = df["candidate"].str.replace('.', '', regex=True)


In [23]:
def cleanCand(x):
    if ', ' in x:
        index = x.index(', ')
        return x[index+2:] + ' ' + x[:index]
    else:
        return x
    
df['candidate'] = df['candidate'].apply(cleanCand)

In [24]:
def cleanCand2(x):
    if ', ' in x:
        index = x.index(', ')
        return x[index+2:] + ' ' + x[:index]
    elif 'DAVID LÓPEZ-NEGRETE' in x:
        return 'DAVID LOPEZ-NEGRETE'
    elif 'CECELIA GONZÁLEZ' in x:
        return 'CECELIA GONZALEZ'
    else:
        return x
    
df['candidate'] = df['candidate'].apply(cleanCand2)

In [25]:
def cleanOffice(x): 
    if ',' in x:
        if '(' not in x:
            index = x.index(',')
            return x[:index]
        else: 
            index = x.index('(')
            return x[:index-1]
    else:
        if 'School' in x:
            return 'SCHOOL BOARD OF TRUSTEES'
        elif 'President' in x:
            return 'US PRESIDENT'
        elif 'County Commissioner' in x or 'COUNTY COMMISSIONER' in x:
            return 'COUNTY COMMISSIONER'
        elif 'Board' in x:
            if 'Advisory' in x:
                return 'TOWN ADVISORY BOARD'
            else:
                return 'TOWN BOARD'
        elif 'Mayor' in x or 'MAYOR' in x:
            return 'MAYOR'
        else:
            return x.upper()

df['office'] = df['race'].apply(cleanOffice)

In [26]:
def cleanOffice2(x): 
    if 'U.S. Representative in Congress' in x:
            x = 'US HOUSE'
    if 'State Assembly' in x:
        x = 'STATE HOUSE'
    if 'SEAT' in x: 
        i = x.index('SEAT')
        if '-' in x: 
            j = x.index('-')
            x = x[:j-1]
        else: 
            x = x[:i-1]
    if 'DEPARTMENT' in x: 
        k = x.index("DEPARTMENT")
        x = x[:k-1]
    if '- 4 YEAR' in x: 
        l = x.index("- 4 YEAR")
        x = x[:l-1]
    else:
        return x.upper()
    return x.upper()
      
df['office'] = df['office'].apply(cleanOffice2)
df = df.drop(['race'], axis=1)

In [27]:
df["state"] = 'NEVADA'
df["state_po"] = 'NV'
df["state_fips"] = '32'
df["state_cen"] = '88'
df["state_ic"] = '65'

In [28]:
df["stage"] = "GEN"
df["date"] = "2020-11-03"
df["year"] = "2020"

df["mode"] = "TOTAL"

df["special"] = "FALSE"
df["writein"] = "FALSE"

In [29]:
# After county name fix, append on fips codes
fips = pd.read_csv('/Users/jennifermiranda/Documents/GitHub/2020-precincts/help-files/county-fips-codes.csv')

fips['state'] = fips['state'].str.upper()
fips = fips.applymap(str)
df['county_name'] = df['county_name'].str.upper()
df=df.applymap(lambda x: x.strip() if type(x)==str else x)
df = pd.merge(df, fips, on = ['state','county_name'], 
             how = 'left')

In [30]:
df['jurisdiction_name'] = df['county_name']
df['jurisdiction_fips'] = df['county_fips']

In [31]:
def getDataverse(x):
    if 'US PRESIDENT' in x:
        return "PRESIDENT"
    elif "US HOUSE" in x:
        return "HOUSE"
    elif 'STATE ' in x:
        return 'STATE'
    elif 'BALLOTS' in x or 'REGISTERED' in x:
        return ''
    elif 'SUPREME COURT JUSTICE' in x or 'STATE SENATE' in x or 'COURT OF APPEALS JUDGE' in x or 'DISTRICT COURT JUDGE' in x or 'UNIVERSITY BOARD OF REGENTS' in x:
        return 'STATE'
    else: 
        return 'LOCAL'
    
df['dataverse'] = df['office'].apply(getDataverse)
df['dataverse'] = df['dataverse'].str.replace('\s+', ' ')

In [32]:
#QA Corrections 
df.loc[df['office'].str.contains('SCHOOL'), 'office'] = 'SCHOOL BOARD OF TRUSTEES'
df.loc[df['office'].str.contains('COMMISSION'), 'office'] = 'COUNTY COMMISSIONER'
df["office"] = df["office"].str.replace('COUNCILMEMBER', 'CITY COUNCIL')
df.loc[df['votes'] == '*', 'votes'] = -1
df.loc[df['office'].str.contains('JUSTICE OF THE PEACE NORTH LAS VEGAS TWNSH'), 'district'] = 'NORTH LAS VEGAS TWNSH TOWNSHIP, DEPARTMENT 3'
df.loc[df['office'].str.contains('JUSTICE OF THE PEACE NORTH LAS VEGAS TWNSH'), 'office'] = 'JUSTICE OF THE PEACE'
df.loc[df['office'].str.contains('JUSTICE OF THE PEACE PAHRUMP'), 'district'] = 'PAHRUMP TOWNSHIP, DEPARTMENT B'
df.loc[df['office'].str.contains('JUSTICE OF THE PEACE PAHRUMP'), 'office'] = 'JUSTICE OF THE PEACE'


In [33]:
#QA Corrections #2
# Readme Check 
df["readme_check"] = "FALSE"
df.loc[df['votes'] == -1, 'readme_check'] = 'TRUE'
df['district'] = df['district'].replace({'1': '001',
                                        '2': '002',
                                        '3': '003',
                                        '4': '004',
                                        '5': '005'})

In [34]:
def QACleanDis(x):
    if 'District' in x:
        comma = x.index(',')
        space = x.index(' ')
        if comma == 10:
            return '00' + x[space+1:comma+1] + x[comma+1:]
        else: 
            return '0' + x[space+1:comma+1] + x[comma+1:]
    else:
        return x

df['district'] = df['district'].apply(QACleanDis)
df.district = df.district.str.replace(' +', ' ')
df["district"] = df["district"].str.upper()


In [35]:
df = df.drop_duplicates()

In [36]:
# Final step: Remove all trailing white space and put columns in correct order. 
df=df.applymap(lambda x: x.strip() if type(x)==str else x)

df=df[["precinct", "office", "party_detailed", "party_simplified", "mode", "votes", "county_name", "county_fips", "jurisdiction_name",
                      "jurisdiction_fips", "candidate", "district", "dataverse", "year", "stage", "state", "special", "writein", "state_po",
                      "state_fips", "state_cen", "state_ic", "date", "readme_check", "magnitude"]]

df=df.set_index('precinct')

df.to_csv('2020-nv-precinct-general.csv')

In [35]:
df

,office,party_detailed,party_simplified,mode,votes,county_name,county_fips,jurisdiction_name,jurisdiction_fips,candidate,...,state,special,writein,state_po,state_fips,state_cen,state_ic,date,readme_check,magnitude
precinct,,,,,,,,,,,,,,,,,,,,,
101,BOARD OF SUPERVISORS,NONPARTISAN,NONPARTISAN,TOTAL,532,CARSON CITY,32510,CARSON CITY,32510,"MAURICE ""MO"" WHITE",...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
101,BOARD OF SUPERVISORS,NONPARTISAN,NONPARTISAN,TOTAL,467,CARSON CITY,32510,CARSON CITY,32510,STACIE WILKE-MCCULLOCH,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
101,COURT OF APPEALS JUDGE,NONPARTISAN,NONPARTISAN,TOTAL,402,CARSON CITY,32510,CARSON CITY,32510,BONNIE BULLA,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
101,COURT OF APPEALS JUDGE,NONPARTISAN,NONPARTISAN,TOTAL,367,CARSON CITY,32510,CARSON CITY,32510,SUSAN BUSH,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
101,COURT OF APPEALS JUDGE,NONPARTISAN,NONPARTISAN,TOTAL,230,CARSON CITY,32510,CARSON CITY,32510,NONE OF THESE CANDIDATES,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Precinct 9,SUPREME COURT JUSTICE,NONPARTISAN,NONPARTISAN,TOTAL,57,WHITE PINE,32033,WHITE PINE,32033,DOUGLAS HERNDON,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
Precinct 9,SUPREME COURT JUSTICE,NONPARTISAN,NONPARTISAN,TOTAL,15,WHITE PINE,32033,WHITE PINE,32033,NONE OF THESE CANDIDATES,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1
Precinct 9,US HOUSE,DEMOCRAT,DEMOCRAT,TOTAL,68,WHITE PINE,32033,WHITE PINE,32033,STEVEN A HORSFORD,...,NEVADA,FALSE,FALSE,NV,32,88,65,2020-11-03,FALSE,1


In [37]:
pd.set_option('max_columns', None)

In [36]:
index = df[df.office.str.contains('JUSTICE OF THE PEACE')]
index

,county_name,precinct,candidate,votes,district,party_detailed,party_simplified,magnitude,office,state,...,stage,date,year,mode,special,writein,county_fips,jurisdiction_name,jurisdiction_fips,dataverse
2074,CLARK,1038,SHANON CLOWERS-SANBORN,52,LAS VEGAS TWNSH DEPARTMENT 12,NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32003,CLARK,32003,LOCAL
2075,CLARK,1038,DIANA L SULLIVAN,237,LAS VEGAS TWNSH DEPARTMENT 12,NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32003,CLARK,32003,LOCAL
2208,CLARK,1039,SHANON CLOWERS-SANBORN,112,LAS VEGAS TWNSH DEPARTMENT 12,NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32003,CLARK,32003,LOCAL
2209,CLARK,1039,DIANA L SULLIVAN,502,LAS VEGAS TWNSH DEPARTMENT 12,NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32003,CLARK,32003,LOCAL
2342,CLARK,1063,SHANON CLOWERS-SANBORN,206,LAS VEGAS TWNSH DEPARTMENT 12,NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32003,CLARK,32003,LOCAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168460,NYE,PAHRUMP 30,KENT JASPERSON,645,"PAHRUMP TOWNSHIP, DEPARTMENT B",NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32023,NYE,32023,LOCAL
168517,NYE,PAHRUMP 31,WALT GRUDZINSKI,399,"PAHRUMP TOWNSHIP, DEPARTMENT B",NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32023,NYE,32023,LOCAL
168518,NYE,PAHRUMP 31,KENT JASPERSON,769,"PAHRUMP TOWNSHIP, DEPARTMENT B",NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32023,NYE,32023,LOCAL
168576,NYE,PAHRUMP 32,WALT GRUDZINSKI,545,"PAHRUMP TOWNSHIP, DEPARTMENT B",NONPARTISAN,NONPARTISAN,1,JUSTICE OF THE PEACE,NEVADA,...,GEN,2020-11-03,2020,TOTAL,FALSE,FALSE,32023,NYE,32023,LOCAL


In [37]:
for x in index['district'].unique():
    #if 'PEACE' in x: 
        print(x)

LAS VEGAS TWNSH DEPARTMENT 12
NORTH LAS VEGAS TWNSH TOWNSHIP, DEPARTMENT 3
PAHRUMP TOWNSHIP, DEPARTMENT B


In [34]:
for x in df['district'].unique():
    #if x == -1: 
        print(x)

WARD 2
DEPARTMENT 3
001, DEPARTMENT 1
001, DEPARTMENT 2
STATEWIDE
007
040
SEAT B
SEAT D
002

001
003
010
038
A
008, DEPARTMENT 1
008, DEPARTMENT 2
008, DEPARTMENT 3
008, DEPARTMENT 4
008, DEPARTMENT 5
008, DEPARTMENT 6
008, DEPARTMENT 7
008, DEPARTMENT 8
008, DEPARTMENT 9
008, DEPARTMENT 10
008, DEPARTMENT 11
008, DEPARTMENT 12
008, DEPARTMENT 13
008, DEPARTMENT 14
008, DEPARTMENT 15
008, DEPARTMENT 16
008, DEPARTMENT 17
008, DEPARTMENT 18
008, DEPARTMENT 19
008, DEPARTMENT 20
008, DEPARTMENT 21
008, DEPARTMENT 22
008, DEPARTMENT 23
008, DEPARTMENT 24
008, DEPARTMENT 25
008, DEPARTMENT 26
008, DEPARTMENT 27
008, DEPARTMENT 28
008, DEPARTMENT 29
008, DEPARTMENT 30
008, DEPARTMENT 31
008, DEPARTMENT 32
008, FAMILY DIVISION, DEPARTMENT A
008, FAMILY DIVISION, DEPARTMENT B
008, FAMILY DIVISION, DEPARTMENT C
008, FAMILY DIVISION, DEPARTMENT D
008, FAMILY DIVISION, DEPARTMENT E
008, FAMILY DIVISION, DEPARTMENT F
008, FAMILY DIVISION, DEPARTMENT G
008, FAMILY DIVISION, DEPARTMENT H
008, FAMIL

In [ ]:
change candidate party:
WASHINGTON, DAVID L. : NO POLITICAL PARTY -> INDEPENDENT
ZAIDI, SAYED "SM" : NO POLITICAL PARTY -> INDEPENDENT
GERHARDT, JOHN CARDIFF : NO POLITICAL PARTY -> INDEPENDENT

In [156]:

for item in index['candidate'].unique():
    if item != 'YES' and item != 'NO':
        print(item)


NO
YES
NOLL, JR., JAKE
YOUNG, MIKE
MILLER, GRATTON
MORRIS, CLAIR
SIMONS, MANDY
SMITH, ROWENA
JOHNSTON, MARGARET A.
KINNEY, CAMERON
GOOLSBY, JONATHAN
MICHELI, CHRIS
MOORE DELRIO, LAURA "NIKI"
SAVEDRA, CARMEN "BETH"
SCHWAB, JAMES L.
SWETT, NOAH J.
THORNAL, SHAUNIA
ZANDER, ERNEST "SKIPP"
DURHAM, JEFF
FLORES, NICOLAS "NICK"
CORONA, DANIEL
KATSONIS, MICHAEL "MIKE"
WINSOR, DELON "DE"
KEYES, RALPH M.
KIRBY, WILLIAM C.
GOICOECHEA, J.J.
CLINTON, ADRIENNE
COATS, PAM
HORKY, GARY
MARBLE, CECELIA
BREWSTER, TODD
MARTIN, LESLIE
MCKAY, HALEY
ROBISON, WADE
BLADES, KRYSTAL
KUEHNERT, SHARYL
36


In [4]:
df

,county_name,precinct,race,candidate,votes
2,Carson City,101,"Board of Supervisors, Ward 2","WHITE, MAURICE ""MO""",532
3,Carson City,101,"Board of Supervisors, Ward 2","WILKE-MCCULLOCH, STACIE",467
4,Carson City,101,"Court Of Appeals Judge, Dept. 3","BULLA, BONNIE",402
5,Carson City,101,"Court Of Appeals Judge, Dept. 3","BUSH, SUSAN",367
6,Carson City,101,"Court Of Appeals Judge, Dept. 3",None Of These Candidates,230
...,...,...,...,...,...
182866,White Pine,Precinct 9,"Supreme Court Justice, Seat D",None Of These Candidates,15
182867,White Pine,Precinct 9,"U.S. Representative in Congress, Dist. 4","Esteban, Jonathan Royce",1
182868,White Pine,Precinct 9,"U.S. Representative in Congress, Dist. 4","Horsford, Steven A.",68
182869,White Pine,Precinct 9,"U.S. Representative in Congress, Dist. 4","Marchant, Jim",64


In [8]:
for item in df['candidate'].unique():
    print(item)

WHITE, MAURICE "MO"
WILKE-MCCULLOCH, STACIE
BULLA, BONNIE
BUSH, SUSAN
None Of These Candidates
RUSSELL, JAMES TODD
WILSON, JIM
BIDEN, JOSEPH R.
BLANKENSHIP, DON
JORGENSEN, JO
TRUMP, DONALD J.
CACIOPPO, JOE
TRUSHENSKI, JOY R.
LOYD, SENA
O'Neill, Philip "PK"
No
Yes
Pickering, Kristina
FUMO, Ozzie
HERNDON, DOUGLAS
ACKERMAN, PATRICIA
Amodei, Mark E.
JANINE HANSEN
MALKOVICH, GREGG
PINDER, PHILIP
STRASDIN, TRICIA
TISDALE, LEE
WHITAKER, KATHRYN
HEATH, JUSTIN
KELLY, KELLI
KOENIG, GREGORY S.
STOCKARD, THOMAS
TITUS, ROBIN L.
Naft, Michael
Thomas, Michael
Villani, Jacob
Yeager, Bita "Marie"
Kierny, Carli Lynn
Scotti, Richard
Ganz, Adam
Trujillo, Monica
Aurbach, Phil
Krall, Nadia
Barisich, Veronica
Coffing, Terry
Bluth, Jacqueline
Leventhal, Todd M.
Bell, Linda Marie
Atkin, Trevor
Peterson, Jessica K.
Silva, Cristina D.
Jones, Tierra
Gonzalez, Elizabeth "Betsy"
Leavitt, Michelle
Denton, Mark
Escobar, Adriana
Hardy, Jr., Joe
Williams, Tim
Albertson, Anna
Villani, Michael
Holthus, Mary Kay
Hunt, Joh

In [18]:
for item in df['office'].unique():
    if '-' in item:
        print(item)

MEMBER MOAPA VALLEY WATER DISTRICT OVERTON - 4 YEAR
MEMBER MOAPA VALLEY WATER DISTRICT LOGANDALE - 4 YEAR
MINDEN-GARDNERVILLE SANITATION DISTRICT
HOSPITAL BOARD TRUSTEE - SEAT A
SCHOOL DISTRICT TRUSTEE - SEAT E
WC - 1


In [54]:
lst = ['School', 'President', 'County Commissioner', 'COUNTY COMMISSIONER', 'QUESTION', 'Board']
lst2 = []
for x in df['race'].unique():
    if ',' not in x: 
        if 'School' not in x and 'Board' not in x and 'COUNTY COMMISSIONER' not in x and 'President' not in x and 'QUESTION' not in x and 'County Commissioner' not in x: 
                lst2.append(x)



In [55]:
lst2

['Justice of the Peace North Las Vegas Twnsh Department 3',
 'Member Moapa Valley Water District Overton - 4 Year',
 'City of Mesquite Councilmember Seat 5',
 'Mayor of Mesquite',
 'Overton Power District 5 Mesquite',
 'Member Moapa Valley Water District Logandale - 4 Year',
 'Gardnerville Ranchos General Improvement District',
 'Minden-Gardnerville Sanitation District',
 'Indian Hills General Improvement District',
 'Round Hill General Improvement District',
 'WEST WENDOVER MAYOR',
 'COUNTY TREASURER',
 'EUREKA COUNTY TELEVISION DISTRICT SEAT 4B',
 'EUREKA COUNTY TELEVISION DISTRICT SEAT 4E',
 'HOSPITAL BOARD TRUSTEE - SEAT A',
 'SCHOOL DISTRICT TRUSTEE - SEAT E',
 'FERNLEY SWIMMING POOL DISTRICT DIRECTOR',
 'NORTH LYON COUNTY FIRE DISTRICT DIRECTOR',
 'JUSTICE OF THE PEACE PAHRUMP DEPARTMENT B',
 'LOVELOCK MEADOWS WATER DISTRICT TRUSTEE',
 'CITY OF LOVELOCK MAYOR',
 'COUNTY RECORDER',
 'Palomino Valley General Improvement District',
 'WC - 1',
 'Incline Village General Improvement Di

In [57]:
for x in lst2:
    if 'Mayor' in x or 'MAYOR' in x:
        print(x)

Mayor of Mesquite
WEST WENDOVER MAYOR
CITY OF LOVELOCK MAYOR


In [24]:
for item in df['race'].unique():
    if ',' in item and '(' in item:
        print(item)

TOPAZ RANCH ESTATES GENERAL IMPROVEMENT DISTRICT (One, 4 Year Term and One 2 Year-Unexpired Term)
SKYLAND GENERAL IMPROVEMENT DISTRICT (Three, 4 Year Terms and Two, 2 Year-Unexpired Terms)


In [23]:
lst =[]
for item in df['race'].unique():
    if ',' in item: 
        if '(' not in item: 
            index = item.index(',')
            print(item[:index])
        else: 
            index = item.index('(')
            print(item[:index-1])
        


Board of Supervisors
Court Of Appeals Judge
District Court Judge
District Court Judge
School Trustee
State Assembly
Supreme Court Justice
Supreme Court Justice
U.S. Representative in Congress
County Commissioner
COUNTY COMMISSIONER
District Court Judge
State Assembly
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Judge
District Court Jud

In [16]:
def unique(list1):
 
    # intilize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    # print list
    for x in unique_list:
        print(x)

In [18]:
unique(lst)

Board of Supervisors
Court Of Appeals Judge
District Court Judge
School Trustee
State Assembly
Supreme Court Justice
U.S. Representative in Congress
County Commissioner
COUNTY COMMISSIONER
Justice of the Peace
State Board of Education
State Senate
University Board of Regents
DOUGLAS COUNTY COMMISSIONER
DOUGLAS COUNTY SCHOOL BOARD
EAST FORK FIRE PROTECTION DISTRICT
TOPAZ RANCH ESTATES GENERAL IMPROVEMENT DISTRICT (One
SKYLAND GENERAL IMPROVEMENT DISTRICT (Three
Elko City Council
SCHOOL BOARD OF TRUSTEES
Carlin City Council
WELLS CITY COUNCIL
WEST WENDOVER CITY COUNCIL
SCHOOL DISTRICT TRUSTEE
CITY COUNCIL
COUNCILMEMBER
NYE COUNTY SCHOOL BOARD TRUSTEE
PERSHING COUNTY SCHOOL DISTRICT TRUSTEE
COUNTY COMMISSION
SCHOOL BOARD TRUSTEE
RENO CITY COUNCIL
Reno City Council
RENO MUNICIPAL COURT JUDGE
School Board Trustee
Sparks City Council
HOSPITAL DISTRICT TRUSTEE


In [129]:
count = 0
notin = 0
total = 0 
for x in cand['candidate']:
    if x in df['candidate'].unique():
        count+=1
    else:
        notin+=1
        print(x)
    total+=1
print(count)
print(notin)
print(total)

GIOCOECHEA, JJ
PAUL R. SCHLEHR
519
2
521


In [ ]:
PAUL R. SCHLEHR
GIOCOECHEA, JJ